"# 🌌 SSZ Calculation Suite\n",
"\n",
"**Segmented Spacetime (SSZ) Physics Calculator**\n",
"\n",
"---\n",
"\n",
"## License: Anti-Capitalist Software License v1.4\n",
"\n",
"© 2025 **Carmen Wrede** & **Lino Casu** | mail@error.wtf\n",
"\n",
"This is anti-capitalist software, released for free use by individuals and organizations that do not operate by capitalist principles.\n",
"\n",
"**Permitted users:** Individuals, non-profits, educational institutions, worker-owned cooperatives, organizations < $1M annual expenses.\n",
"\n",
"---\n",
"\n",
"This notebook demonstrates the core SSZ calculations:\n",
"- Segment density Ξ(r) in weak and strong field regimes\n",
"- Time dilation D_SSZ vs D_GR\n",
"- Gravitational redshift with Δ(M) correction\n",
"- Power law energy normalization"

In [ ]:
# Install dependencies
!pip install -q numpy pandas plotly

In [ ]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Constants
G = 6.67430e-11      # m³/(kg·s²)
c = 299792458.0      # m/s
M_SUN = 1.98847e30   # kg
PHI = 1.6180339887498948  # Golden ratio

# Δ(M) correction parameters
A_DM = 98.01
ALPHA_DM = 2.7177e4
B_DM = 1.96

print("✅ SSZ Constants loaded")
print(f"   φ (Golden Ratio) = {PHI:.6f}")
print(f"   G = {G:.5e} m³/(kg·s²)")
print(f"   c = {c:.0f} m/s")

## 📐 Core SSZ Functions

In [ ]:
def schwarzschild_radius(M_kg):
    """Calculate Schwarzschild radius r_s = 2GM/c²"""
    return 2 * G * M_kg / (c * c)

def xi_weak(r, r_s):
    """Weak field segment density: Ξ = r_s/(2r)"""
    return r_s / (2 * r)

def xi_strong(r, r_s, phi=PHI):
    """Strong field segment density: Ξ = 1 - exp(-φr/r_s)"""
    return 1.0 - np.exp(-phi * r / r_s)

def D_ssz(r, r_s, mode='auto'):
    """SSZ time dilation: D = 1/(1+Ξ)"""
    x = r / r_s
    if mode == 'weak' or (mode == 'auto' and x > 110):
        xi = xi_weak(r, r_s)
    else:
        xi = xi_strong(r, r_s)
    return 1.0 / (1.0 + xi)

def D_gr(r, r_s):
    """GR time dilation: D = √(1 - r_s/r)"""
    if r <= r_s:
        return 0.0
    return np.sqrt(1.0 - r_s / r)

def z_gr(M_kg, r_m):
    """GR gravitational redshift: z = 1/√(1-r_s/r) - 1"""
    r_s = schwarzschild_radius(M_kg)
    if r_m <= r_s:
        return float('nan')
    return 1.0 / np.sqrt(1.0 - r_s / r_m) - 1.0

def delta_m_correction(M_kg):
    """Δ(M) φ-based correction from spiral geometry"""
    r_s = schwarzschild_radius(M_kg)
    lM = np.log10(M_kg) if M_kg > 0 else 30.0
    norm = min(1.0, max(0.0, (lM - 10.0) / 32.0))
    return (A_DM * np.exp(-ALPHA_DM * r_s) + B_DM) * norm

def z_ssz(M_kg, r_m):
    """
    SSZ gravitational redshift.
    
    CRITICAL: z_SSZ = z_GR × (1 + Δ(M)/100)
    From 'Dual Velocities' paper: SSZ matches GR with small φ-correction
    """
    z_g = z_gr(M_kg, r_m)
    delta_m = delta_m_correction(M_kg)
    return z_g * (1.0 + delta_m / 100.0)

print("✅ SSZ functions defined")

## 🧪 Example: Neutron Star Calculation

In [ ]:
# PSR J0740+6620 - Heavy neutron star
M_msun = 2.08
R_km = 13.7

M_kg = M_msun * M_SUN
R_m = R_km * 1000
r_s = schwarzschild_radius(M_kg)

print("="*50)
print("PSR J0740+6620 Analysis")
print("="*50)
print(f"Mass:     {M_msun} M☉")
print(f"Radius:   {R_km} km")
print(f"r_s:      {r_s/1000:.3f} km")
print(f"r/r_s:    {R_m/r_s:.2f}")
print("-"*50)

# Calculate
xi = xi_strong(R_m, r_s)
d_ssz = D_ssz(R_m, r_s)
d_gr = D_gr(R_m, r_s)
z_g = z_gr(M_kg, R_m)
z_s = z_ssz(M_kg, R_m)
delta = delta_m_correction(M_kg)

print(f"Ξ(R):     {xi:.4f}")
print(f"D_SSZ:    {d_ssz:.4f}")
print(f"D_GR:     {d_gr:.4f}")
print("-"*50)
print(f"z_GR:     {z_g:.4f}")
print(f"z_SSZ:    {z_s:.4f}")
print(f"Δ(M):     {delta:.2f}%")
print(f"Diff:     {(z_s-z_g)/z_g*100:.2f}%")
print("="*50)

## 📊 Plot: SSZ vs GR Time Dilation

In [ ]:
r_s = 3000  # Reference Schwarzschild radius
r_range = np.linspace(1.01, 10, 200) * r_s

d_ssz_vals = [D_ssz(r, r_s, mode='strong') for r in r_range]
d_gr_vals = [D_gr(r, r_s) for r in r_range]

fig = go.Figure()

fig.add_trace(go.Scatter(
    x=r_range/r_s, y=d_gr_vals,
    mode='lines', name='D_GR (General Relativity)',
    line=dict(color='#3498db', width=3)
))

fig.add_trace(go.Scatter(
    x=r_range/r_s, y=d_ssz_vals,
    mode='lines', name='D_SSZ (Segmented Spacetime)',
    line=dict(color='#e74c3c', width=3)
))

# Mark horizon
fig.add_vline(x=1, line_dash="dot", line_color="gray")
fig.add_annotation(x=1.1, y=0.55, text="D_SSZ(r_s)=0.555\n(FINITE!)", 
                   showarrow=True, arrowhead=2, font=dict(color='#e74c3c'))
fig.add_annotation(x=1.1, y=0.05, text="D_GR→0\n(SINGULARITY)",
                   showarrow=True, arrowhead=2, font=dict(color='#3498db'))

fig.update_layout(
    title="SSZ vs GR: Time Dilation Comparison",
    xaxis_title="r/r_s",
    yaxis_title="D(r) = dτ/dt",
    height=500,
    showlegend=True
)

fig.show()

## 📊 Plot: Segment Density Ξ(r)

In [ ]:
r_s = 3000
r_strong = np.linspace(0.5, 5, 100) * r_s
r_weak = np.linspace(5, 200, 100) * r_s

xi_s = [xi_strong(r, r_s) for r in r_strong]
xi_w = [xi_weak(r, r_s) for r in r_weak]

fig = go.Figure()

fig.add_trace(go.Scatter(
    x=r_strong/r_s, y=xi_s,
    mode='lines', name='Ξ Strong: 1-exp(-φr/r_s)',
    line=dict(color='#e74c3c', width=3)
))

fig.add_trace(go.Scatter(
    x=r_weak/r_s, y=xi_w,
    mode='lines', name='Ξ Weak: r_s/(2r)',
    line=dict(color='#3498db', width=3)
))

# Mark key point
xi_at_horizon = xi_strong(r_s, r_s)
fig.add_trace(go.Scatter(
    x=[1], y=[xi_at_horizon],
    mode='markers', name=f'Ξ(r_s)={xi_at_horizon:.3f}',
    marker=dict(size=15, color='gold', symbol='star')
))

fig.update_layout(
    title="Segment Density Ξ(r) - Weak vs Strong Field",
    xaxis_title="r/r_s",
    yaxis_title="Ξ(r)",
    xaxis_type="log",
    height=450
)

fig.show()

## 🔬 Batch Calculation: Multiple Objects

In [ ]:
# Define objects
objects = [
    ("Sun", 1.0, 696340),
    ("Sirius B (WD)", 1.02, 5800),
    ("PSR J0030+0451", 1.44, 13.0),
    ("PSR J0348+0432", 2.01, 13.0),
    ("PSR J0740+6620", 2.08, 13.7),
    ("Cygnus X-1", 21.0, 44.0),
]

results = []
for name, M_msun, R_km in objects:
    M_kg = M_msun * M_SUN
    R_m = R_km * 1000
    r_s = schwarzschild_radius(M_kg)
    
    z_g = z_gr(M_kg, R_m)
    z_s = z_ssz(M_kg, R_m)
    delta = delta_m_correction(M_kg)
    
    x = R_m / r_s
    if x < 2:
        regime = "very_close"
    elif x <= 3:
        regime = "photon_sphere"
    elif x <= 10:
        regime = "strong"
    else:
        regime = "weak"
    
    results.append({
        "Name": name,
        "M (M☉)": M_msun,
        "R (km)": R_km,
        "r/r_s": f"{x:.1f}",
        "Regime": regime,
        "z_GR": f"{z_g:.4f}",
        "z_SSZ": f"{z_s:.4f}",
        "Δ(M) %": f"{delta:.2f}"
    })

df = pd.DataFrame(results)
df

## 📚 References

1. Wrede, C., Casu, L., Bingsi (2025). *Dual Velocities in Segmented Spacetime*. ResearchGate.
2. Wrede, C., Casu, L. (2025). *Segmented Spacetime - Mass Projection & Unified Results*. GitHub.
3. Wrede, C., Casu, L. (2025). *Segmented Spacetime and the Natural Boundary of Black Holes*.

---

**Contact:** mail@error.wtf  
**License:** Anti-Capitalist Software License v1.4